# Premier pipeline résumé

vérification que chaque étape marche, donc plus détaillé, avec chaque étape de fonction

In [ ]:
!pip install gensim==3.8.3
!pip install unidecode
!pip install torch
!pip install sentencepiece
!pip install transformers==4.5.0

In [1]:
from joblib.parallel import cpu_count
import pandas as pd
import numpy as np
import torch
import pickle
import time
import functools
import operator
from joblib import Parallel,delayed
from functools import partial
import sentencepiece as spm 
import psutil
from tqdm import tqdm
import pickle
import os
import re
from unidecode import unidecode
from bs4 import BeautifulSoup
import gensim

from fats import BERTScore,TextRank,Random_summary,Lead_3,SMHA_Linear_classifier
from fats import Make_Extractive,Word_Cleaning, Make_Embedding

In [2]:
base=pd.read_csv('MLSUM_fr_val.csv',sep=";")
Paragraphes= base.iloc[0,1].split('.') #introduisez ici une suite de phrases non-nettoyées dans une liste

# Paragraphes = [ phrase1, phrase2, ..., phraseN ]
#Paragraphes

In [3]:
WC=Word_Cleaning(n_jobs=1,sentence=True,threshold=True,seuil=2,lemma=False,seuil_carac=3)
Paragraphes=WC.remove_empty(Paragraphes)
text=WC.make_documents(Paragraphes)
text=WC.remove_empty(text)
#text

In [4]:
W2V=gensim.models.Word2Vec.load("W2V_all.model")

from transformers import CamembertTokenizer, CamembertModel, CamembertConfig
tok=CamembertTokenizer("MLSUM_tokenizer.model")

ME=Make_Extractive(cpu=1)
#sentence=ME.make_w2v_sentences(text)
#text=Parallel(n_jobs=cpu_max)(delayed(ME.make_splitting)(s) for s in text)

dico=ME.make_encoding(text,tokenizer='MLSUM_tokenizer.model',voc_size=12000,split=1,training=False)
input_ids=dico['input']
att_mask=dico['mask']
camem=CamembertModel.from_pretrained("camembert-base")
ME=Make_Embedding(tok,cpu=1)
embeddings=ME.emb_phrase(input_ids,att_mask,camem)

In [5]:
from transformers import CamembertTokenizer, CamembertModel, CamembertConfig
tok=CamembertTokenizer("MLSUM_tokenizer.model")
ME=Make_Embedding(tok,cpu=1)

dico=ME.make_tokens(text,cpu=1)
dico.keys()

dict_keys(['input_ids', 'attention_mask'])

In [6]:
def emb_phrase(input_id,att_mask,cam):
    embedding=cam(torch.tensor(input_id).squeeze(1),torch.tensor(att_mask).squeeze(1))
    return embedding.last_hidden_state.mean(dim=0)

In [27]:
x=camem(torch.tensor(dico['input_ids']).squeeze(1),torch.tensor(dico['attention_mask']).squeeze(1))
y=x.last_hidden_state.mean(dim=0)

In [ ]:
y

In [7]:
emb_phrase(dico['input_ids'],dico['attention_mask'],camem)

tensor([[ 0.0227,  0.1509,  0.0521,  ..., -0.0226,  0.0191, -0.0921],
        [ 0.0375,  0.0652, -0.1596,  ...,  0.0345,  0.0453, -0.1225],
        [ 0.0136,  0.1050, -0.0628,  ...,  0.0289,  0.0500, -0.1180],
        ...,
        [-0.0235,  0.1691, -0.0033,  ...,  0.0187, -0.0276, -0.1260],
        [-0.0235,  0.1691, -0.0033,  ...,  0.0187, -0.0276, -0.1260],
        [-0.0235,  0.1691, -0.0033,  ...,  0.0187, -0.0276, -0.1260]],
       grad_fn=<MeanBackward1>)

In [4]:
nphrase=2
TR=TextRank(tok_path='MLSUM_tokenizer.model',cpu=1)
BS=BERTScore('MLSUM_tokenizer.model')

In [5]:
TRB=TR.make_resume(text,'bert',k=nphrase,get_score_only=True)

In [6]:
TRB

[426, 334]

In [ ]:
TRW=TR.make_resume(text,'word2vec',W2V=W2V,k=nphrase)
BSR=BS.make_summary(text,k=nphrase)
L3=Lead_3(text,k=2)
RS=Random_summary(text)